In [2]:
# -*- coding: utf-8 -*-

#module importing
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time



# 2. driver경로설정 및 url주소 설정
driverpath = 'C:/Users/Heewon/driver/chromedriver.exe'
dr = webdriver.Chrome(driverpath)

####

#로그인
dr.get('https://www.kita.net/login/login.do?geturl=http://stat.kita.net/stat/pmsd/china/ChinaWholeList.screen')
dr.find_element_by_id('p_member_id').send_keys('kedkorea12')
dr.find_element_by_id('p_passwd').send_keys('ked2363!')
dr.find_element_by_xpath('''//*[@id="myform"]/fieldset/button''').click()

####

df_tot = pd.DataFrame(columns = ['year','month','country','SC','S_code', 'export', 'import', 'page'])

#한국 URL
naraURL = ['http://stat.kita.net/stat/kts/pum/ItemImpExpList.screen']
nara = ['한국']
for site, 나라 in zip(naraURL, nara) :
    #사이트 진입
    dr.get(site)
    #누계 -> 당월
    dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[4]/select/option[1]''').click()
    #100개씩 보기
    dr.find_element_by_xpath('''//*[@id="listCount"]/option[3]''').click()
    
    #연도 43개
    for year_ in [13, 20] : 
        dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).click()
        년 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).text)
        
        
        #월 12개
        for month_ in range(1,13) : 
            
            if ((년 == 2019) and (month_ == 2)) :
                break;
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).click()
            월 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).text)

                
            #조회
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[3]/a''').click()                    
            time.sleep(0.5)
            
            abc=0
            temp = 0
            
            while 1:
                
                
                try :
                    if dr.find_element_by_xpath('''//*[@id="mySheet1"]/tbody/tr[3]/td/div/table/tbody/tr/td/div''').text == '조회된 데이터가 없습니다.' :
                        break;
                except :
        
                    try :
                        
                        abc+=1
                        drt = dr.page_source
                        sou = BeautifulSoup(drt,'html.parser')
                        soup = sou.find('tbody')

                        #데이터 적재
                        코드_소 = []
                        소분류 = []
                        수출액 = []
                        수출증감률 = []
                        수입액 = []
                        수입증감률 = []
                        수지 = []
                        
                        
                        #첫페이지 
                        #데이터 적재
                        
                        if abc==1:
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2].text)

                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+6].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10+7].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+8].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10+9].text)
                                i+=1
                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        else: 
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        print('1page done')
                        ####################
                        if (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1) == 1:
                            break;
                        
                        for tmp in range(1, (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1)):
                            tmpt = int(dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).text)
                            print(temp, tmpt)
                            if temp > tmpt :
                                slack.chat.post_message('#analysis', '{0}  error'.format(년*100000 +월*1000+tmpt))
                                break;
                            print('{0}  done'.format(년*100000 +월*1000+tmpt))
                            dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).click()
                            time.sleep(0.5)
                            
                            drt = dr.page_source
                            sou = BeautifulSoup(drt,'html.parser')
                            soup = sou.find('tbody')
                            temp+=1
                            #데이터 적재
                            코드_소 = []
                            소분류 = []
                            수출액 = []
                            수출증감률 = []
                            수입액 = []
                            수입증감률 = []
                            수지 = []

                            #데이터 적재
                            i=1
                            print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1
                            

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                            print(df.shape)
                            
                            if df.shape[0] != 100:
                                drt = dr.page_source
                                sou = BeautifulSoup(drt,'html.parser')
                                soup = sou.find('tbody')
                                
                                #데이터 적재
                                코드_소 = []
                                소분류 = []
                                수출액 = []
                                수출증감률 = []
                                수입액 = []
                                수입증감률 = []
                                수지 = []

                                #데이터 적재
                                i=1
                                print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                    코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                    소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                    수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                    #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                    수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                    #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                    #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                    i+=1


                                df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                                print(df.shape)
                            
                            df_tot = pd.concat([df_tot, df])
                        
                        
                        if temp > tmpt :
                             break;
                        print('done')    
                        #화살표 넘기기
                        dr.find_element_by_xpath('''//*[@id="pageArea"]/a[2]''').click()
                        time.sleep(1)
                        

                    except:
                        break;
            
            print('#analysis', '{0}  done'.format(년*100 +월))
            
            df_tot.to_csv('./{0}korea3.csv'.format(년), index=False)

1page done
0 2
200701002  done
38,544
(100, 8)
1 3
200701003  done
17,579
(60, 8)
17,579
(100, 8)
2 4
200701004  done
10,966
(100, 8)
3 5
200701005  done
7,570
(100, 8)
4 6
200701006  done
5,230
(100, 8)
5 7
200701007  done
3,775
(100, 8)
6 8
200701008  done
2,890
(100, 8)
7 9
200701009  done
2,890
(100, 8)
8 10
200701010  done
1,904
(60, 8)
1,904
(100, 8)
done
1page done
9 12
200701012  done
1,237
(100, 8)
10 13
200701013  done
1,026
(100, 8)
11 14
200701014  done
850
(100, 8)
12 15
200701015  done
695
(30, 8)
695
(100, 8)
13 16
200701016  done
588
(90, 8)
588
(100, 8)
14 17
200701017  done
498
(100, 8)
15 18
200701018  done
417
(60, 8)
417
(100, 8)
16 19
200701019  done
347
(100, 8)
17 20
200701020  done
288
(100, 8)
done
1page done
18 22
200701022  done
198
(100, 8)
19 23
200701023  done
163
(100, 8)
20 24
200701024  done
163
(100, 8)
21 25
200701025  done
108
(90, 8)
108
(100, 8)
22 26
200701026  done
89
(100, 8)
23 27
200701027  done
71
(100, 8)
24 28
200701028  done
55
(100, 8)
2

0
(46, 8)
0
(46, 8)
done
1page done
48 51
200704051  done
0
(100, 8)
49 53
200704053  done
0
(100, 8)
50 54
200704054  done
0
(100, 8)
done
1page done
51 51
200704051  done
0
(100, 8)
52 53
200704053  done
0
(30, 8)
0
(100, 8)
53 54
200704054  done
0
(100, 8)
done
1page done
54 51
#analysis 200704  done
1page done
0 2
200705002  done
45,026
(100, 8)
1 3
200705003  done
21,002
(100, 8)
2 4
200705004  done
12,034
(100, 8)
3 5
200705005  done
8,090
(60, 8)
8,090
(100, 8)
4 6
200705006  done
5,778
(100, 8)
5 7
200705007  done
4,191
(100, 8)
6 8
200705008  done
3,270
(100, 8)
7 9
200705009  done
2,615
(100, 8)
8 10
200705010  done
2,136
(90, 8)
2,136
(100, 8)
done
1page done
9 12
200705012  done
1,404
(60, 8)
1,404
(100, 8)
10 13
200705013  done
1,154
(100, 8)
11 14
200705014  done
963
(100, 8)
12 15
200705015  done
773
(100, 8)
13 16
200705016  done
666
(100, 8)
14 17
200705017  done
567
(100, 8)
15 18
200705018  done
479
(100, 8)
16 19
200705019  done
381
(100, 8)
17 20
200705020  done
30

0
(30, 8)
0
(100, 8)
42 48
200708048  done
0
(100, 8)
43 49
200708049  done
0
(100, 8)
44 50
200708050  done
0
(100, 8)
done
1page done
45 52
200708052  done
0
(100, 8)
46 53
200708053  done
0
(100, 8)
47 54
200708054  done
0
(92, 8)
0
(92, 8)
done
1page done
48 51
200708051  done
0
(100, 8)
49 53
200708053  done
0
(90, 8)
0
(100, 8)
50 54
200708054  done
0
(92, 8)
0
(92, 8)
done
1page done
51 51
200708051  done
0
(90, 8)
0
(100, 8)
52 53
200708053  done
0
(90, 8)
0
(100, 8)
53 54
200708054  done
0
(92, 8)
0
(92, 8)
done
1page done
54 51
#analysis 200708  done
1page done
0 2
200709002  done
39,680
(100, 8)
1 3
200709003  done
18,169
(100, 8)
2 4
200709004  done
11,285
(100, 8)
3 5
200709005  done
7,633
(100, 8)
4 6
200709006  done
5,449
(100, 8)
5 7
200709007  done
3,913
(100, 8)
6 8
200709008  done
3,054
(100, 8)
7 9
200709009  done
2,404
(100, 8)
8 10
200709010  done
1,898
(100, 8)
done
1page done
9 12
200709012  done
1,266
(100, 8)
10 13
200709013  done
1,084
(100, 8)
11 14
20070901

0
(100, 8)
38 44
200712044  done
0
(30, 8)
0
(100, 8)
39 45
200712045  done
0
(90, 8)
0
(100, 8)
40 46
200712046  done
0
(100, 8)
41 47
200712047  done
0
(100, 8)
42 48
200712048  done
0
(100, 8)
43 49
200712049  done
0
(100, 8)
44 50
200712050  done
0
(100, 8)
done
1page done
45 52
200712052  done
0
(100, 8)
46 53
200712053  done
0
(100, 8)
47 54
200712054  done
0
(100, 8)
48 55
200712055  done
0
(15, 8)
0
(15, 8)
done
1page done
49 51
200712051  done
0
(100, 8)
50 53
200712053  done
0
(30, 8)
0
(100, 8)
51 54
200712054  done
0
(100, 8)
52 55
200712055  done
0
(15, 8)
0
(15, 8)
done
1page done
53 51
#analysis 200712  done
1page done
0 2
200001002  done
17,892
(60, 8)
17,892
(100, 8)
1 3
200001003  done
8,621
(100, 8)
2 4
200001004  done
5,511
(100, 8)
3 5
200001005  done
3,655
(100, 8)
4 6
200001006  done
2,689
(100, 8)
5 7
200001007  done
2,057
(100, 8)
6 8
200001008  done
1,511
(60, 8)
1,511
(100, 8)
7 9
200001009  done
1,206
(100, 8)
8 10
200001010  done
946
(100, 8)
done
1page don

112
(100, 8)
20 24
200005024  done
91
(100, 8)
21 25
200005025  done
74
(100, 8)
22 26
200005026  done
60
(100, 8)
23 27
200005027  done
49
(100, 8)
24 28
200005028  done
39
(90, 8)
39
(100, 8)
25 29
200005029  done
30
(100, 8)
26 30
200005030  done
24
(100, 8)
done
1page done
27 32
200005032  done
13
(100, 8)
28 33
200005033  done
9
(100, 8)
29 34
200005034  done
6
(100, 8)
30 35
200005035  done
4
(100, 8)
31 36
200005036  done
2
(100, 8)
32 37
200005037  done
0
(100, 8)
33 38
200005038  done
0
(100, 8)
34 39
200005039  done
0
(100, 8)
35 40
200005040  done
0
(100, 8)
done
1page done
36 42
200005042  done
0
(100, 8)
37 43
200005043  done
0
(100, 8)
38 44
200005044  done
0
(100, 8)
39 45
200005045  done
0
(100, 8)
40 46
200005046  done
0
(100, 8)
41 47
200005047  done
0
(30, 8)
0
(100, 8)
42 48
200005048  done
0
(100, 8)
43 49
200005049  done
0
(100, 8)
44 50
200005050  done
0
(100, 8)
done
1page done
#analysis 200005  done
1page done
0 2
200006002  done
22,626
(100, 8)
1 3
200006003  

642
(100, 8)
11 14
200010014  done
537
(100, 8)
12 15
200010015  done
457
(100, 8)
13 16
200010016  done
387
(100, 8)
14 17
200010017  done
322
(100, 8)
15 18
200010018  done
265
(100, 8)
16 19
200010019  done
215
(100, 8)
17 20
200010020  done
185
(60, 8)
185
(100, 8)
done
1page done
18 22
200010022  done
127
(100, 8)
19 23
200010023  done
106
(100, 8)
20 24
200010024  done
89
(100, 8)
21 25
200010025  done
72
(100, 8)
22 26
200010026  done
59
(100, 8)
23 27
200010027  done
48
(100, 8)
24 28
200010028  done
36
(100, 8)
25 29
200010029  done
29
(100, 8)
26 30
200010030  done
23
(100, 8)
done
1page done
27 32
200010032  done
13
(100, 8)
28 33
200010033  done
9
(100, 8)
29 34
200010034  done
6
(100, 8)
30 35
200010035  done
3
(60, 8)
3
(100, 8)
31 36
200010036  done
1
(100, 8)
32 37
200010037  done
0
(100, 8)
33 38
200010038  done
0
(100, 8)
34 39
200010039  done
0
(100, 8)
35 40
200010040  done
0
(100, 8)
done
1page done
36 42
200010042  done
0
(100, 8)
37 43
200010043  done
0
(100, 8)
